In [ ]:
import subprocess
import numpy as np
import math
import matplotlib.pyplot as plt
# Enable interactive plot
%matplotlib notebook
import os
from numpy import linalg as la
import cmath
import random
import sys
import pickle
from scipy.optimize import minimize
from sympy import solve, sqrt, exp
import pandas as pd
import gc
gc.collect()

# Environmental Fitness Function

In [ ]:
# Environmental fitness function with fixed area
def Fitness(x,x0,s):
    #fitness of x in env (x0,s)
    return np.exp(-(x-x0)**2/(2*s**2))/np.sqrt(2*np.pi*s**2)

# Solver for Optimal Phenotype

In [ ]:
# Function to check whether phenotype z is the optimal phenotype. Returns zero when the slopes of the adaptive function
# and the fitness sets match, nonzero value otherwise
def Roots(z,x1,s1,x2,s2,p):
    x = z[0]   
    f = np.empty((1))
    f[0] = p*(x-x1)*s2**2*Fitness(x,x1,s1)+(1-p)*(x-x2)*s1**2*Fitness(x,x2,s2)
    return f


# Solver that returns a matrix with optimal phenotype values as a function of time t in range(0,T), for a given rate of
# environmental change v, and a given p value. 
def optp(v,T,p):
    optimalmat=np.zeros((T))
    opt=(x01+x02)/2

    for t in range(T):
        x1=x01
        x2=x02+v*t         

        # The solver is sensitive to initial guesses, so we compare two solutions: one (zGuess1) where the initial guess
        # is one of the specialists, and another (zGuess2) where the initial guess is the optimal solution in the
        # previous time step. This produces the best results, especially for skewed fitness sets.

        m=(Fitness(x2,x2,s2)-Fitness(x1,x2,s2))/(Fitness(x2,x1,s1)-Fitness(x1,x1,s1)) #slope of line joining the two specialists on the fitness set
        mp=-p/(1-p)    #slope of the adaptive function
        if(mp<=m):
            zGuess1 = np.array([x1])         # if the slope of the adaptive function is smaller that that of the 
        elif(mp>m):                          # line joining the two specialists on the fitness set, choose x1 as the initial guess, 
            zGuess1 = np.array([x2])         # otherwise choose x2

        bounds = [(x1,x2)]                   # only look for solutions in the range(x1,x2)
        
        # Roots(z,x1,s1,x2,s2,p) returns a value f for each z, the following finds z within the defined bounds 
        # that minimizes |f|, starting with some initial guesses for z
        z1 = minimize(lambda z: np.linalg.norm(Roots(z,x1,s1,x2,s2,p)),tol=1e-15, x0=zGuess1, bounds=bounds)
        opt1=z1.x[0]
        A1=p*Fitness(opt1,x1,s1)+(1-p)*Fitness(opt1,x2,s2) # value of Adaptive function at the first solution, opt1

        #repeat for second initial guess
        zGuess2 = np.array([opt])
        bounds = [(x1,x2)]
        z2 = minimize(lambda z: np.linalg.norm(Roots(z,x1,s1,x2,s2,p)),tol=1e-15, x0=zGuess2, bounds=bounds)
        opt2=z2.x[0]
        A2=p*Fitness(opt2,x1,s1)+(1-p)*Fitness(opt2,x2,s2)  # value of Adaptive function at the second solution, opt2

        if(A2>A1):         # chose opt2 as the correct optimal if A2>A1, else choose opt1
            opt=opt2
        else:
            opt=opt1

        optimalmat[t]=opt

    return optimalmat

# Tolerances for The Effective Fitness Function

In [ ]:
# Function that returns the tolerance of the effective fitness function that governs selection as a function of time,
# for a given rate of environmental change v, and for a given p value
def opts(v,T,optimalmat,p):
    sigmamat=np.zeros((T))

    for t in range(T):
        if abs(x01-optimalmat[t])<0.5:
            sigmamat[t]=s1
        elif abs(x02+v*t-optimalmat[t])<0.5:
            sigmamat[t]=s2
        else:
            sigmamat[t]=sqrt(p*s1**2+(1-p)*s2**2)
    return sigmamat

# Population Dynamics: Selection, Mutation & Resizing for Carrying Capacity

In [ ]:
# Population level dynamics in which selection is governed by an effective fitness function which has a maxima at the
# optimal phenotype, and a tolerance obtained from a weighted sum of the two environmental tolerances. Returns 
# whether a population survives for a given value of  v, p, and s2/s1 
def dynamics(v,T,optimalmat,sigmamat):
       
    mu, sigma = optimalmat[0],sigmapop          # mean and standard deviation of the initial phenotypic distribution
    s = np.random.normal(mu, sigma, initsize)   #initialize the population
        
    # Turn the set of phenotypes into a histogram distribution
    binnumber=30  
    dist=np.zeros((binnumber,2))
    hist,bin_edges=np.histogram(s,bins=binnumber)
    dist[:,0]=(bin_edges[1:]+bin_edges[:-1])/2
    dist[:,1]=hist    

    survival=1
    for time in range(T):
        newpheno=np.empty(0)
        newphenosized=np.empty(0)
            
        optimal=optimalmat[time]
        omegaf=sigmamat[time]
        
        # For each phenotype, compute the number of offsprings in the next generation after selection by
        # an effective fitness function, and then add mutations as Gaussian noise with mean zero and variance
        # equal to the mutation rate mutr 
        for i in range(binnumber): #selection
            f=math.floor(dist[i,1]*Fmax*math.exp(-((dist[i,0]-(optimal))**2)/(2*omegaf**2)))
            for p in range (f):
                newpheno=np.append(newpheno, random.gauss(dist[i,0], mutr))
               
        size=len(newpheno)
        if time<T-1 and size<=5:    # Population survives if its size does not fall below 5 till t=T
            survival=0
            #print("Failed at time ",time )
            break    

        if size>initsize:                                             # if population size is greater than 
            newphenosized=random.sample(list(newpheno), initsize)     # carrying capacity K, pick a random  
        else:                                                         # sample of K individuals for to undergo
            newphenosized=newpheno                                    # selection in the next generation
            
        hist,bin_edges=np.histogram(newphenosized,bins=binnumber)     # Turn the resized population after selection  
        dist[:,0]=(bin_edges[1:]+bin_edges[:-1])/2                    # and added mutations into a histogram distribution
        dist[:,1]=hist                                                # that undergoes selection in the next generation
        
    return survival

# Compute Lower Bounds

In [ ]:
offset=0.01              # p=p_c+offset
v0=0.05

Ns=21
sr0=0.5
ds=0.025
s_range=np.zeros(Ns)     # range of tolerance asymmetry ratios s2/s1 for the two environments
for i in range(Ns):
    s_range[i]=sr0+i*ds
print(s_range)

pc_vals=1/(1+s_range)    # p_c=1/(1+s2/s1)


s1=10                  # Tolerance of the stationary environment
s2=0                   # Initialize the tolerance of the moving environment
seff=0                 # Initialize the variance of the effective fitness function that governs selection
sigmapop=0             # Initialize the variance of initial phenotypic distribution
T0=T=1200              # Maximum simulation duration
x01=100                # Initial phenotypic value of stationary specialist
x02=110                # Initial phenotypic value of moving specialist
initsize=1000          # carrying capacity K
mutr=0.5               # mutation rate
Fmax=2                 # maximum number of offsprings

Niter=25               # Number of populations to study for each and s2/s1

vc_low=np.zeros((Ns,Niter))
vc_high=np.zeros((Ns,Niter))

dir1='DATA/vc_lims_mutr{}_off{}/'.format(mutr,offset)
if not os.path.exists(dir1):
    os.makedirs(dir1)

f1=open(dir1+'vlims.txt',"w")
tol1=1e-4
tol2=1e-2

print("##################### offset = {} #####################".format(offset))
for i in range(Ns):
    p=pc_vals[i]+offset
    sr=s_range[i]
    print("------- sr = {}, pc = {}, -------".format(sr,p))
    s2=sr*s1
    seff=math.sqrt(p*s1**2+(1-p)*s2**2)   # tolerance of effective fitness function
    sigmapop=math.sqrt(mutr**2+mutr*math.sqrt(mutr**2+4*seff**2))/2     # variance of initial phenotypic distribution
    for r in range(Niter): 
        
        # Compute upper bound using a binary search algorithm
        start=0.0
        end=1.0
        v_range=end-start
        while(v_range>tol1):          
            v=start+(end-start)/2                
            T=int(T0*v0/v)+50 
            optimalmat=optp(v,T,p)
            sigmamat=opts(v,T,optimalmat,p)
            
            survival=dynamics(v,T,optimalmat,sigmamat)
            if(survival):
                start=v
            else:
                end=v
            v_range=end-start
            if(start==0 and end<tol2):
                start=v
                break
        vc_high[i,r]=start

        # Compute lower bound using a binary search algorithm
        if vc_high[i,r]<tol2:
            vc_low[i,r]=vc_high[i,r]
        else:
            start=0
            end=vc_high[i,r]
            v_range=end-start
            while(v_range>tol1):
                v=start+(end-start)/2                
                T=int(T0*v0/v)             
                optimalmat=optp(v,T,p)
                sigmamat=opts(v,T,optimalmat,p)

                survival=dynamics(v,T,optimalmat,sigmamat)
                if(survival):
                    end=v
                else:
                    start=v
                v_range=end-start
                if(start==0 and end<tol2):
                    start=v
                    break
            vc_low[i,r]=end
        print(' Iteration : {}, vc_low = {}, vc_high = {}'.format(r,vc_low[i,r],vc_high[i,r]))
        f1.write("{} {} {} {} {}\n".format(s_range[i],r,vc_low[i,r],vc_high[i,r],p))
        f1.flush()
            
f1.close()

